In [50]:
pip install arxiv scholarly requests


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
import arxiv

arXiv **Don't have Cited** cc ChatGPT

In [ ]:

query = "Computer Science"
search = arxiv.Search(
    query=query,
    max_results=10,  # Number of results to fetch
    sort_by=arxiv.SortCriterion.Relevance
)


for result in search.results():
    print(f"Title: {result.title}")
    print(f"Authors: {[author.name for author in result.authors]}")
    print(f"Summary: {result.summary}")
    print(f"Published: {result.published}")
    print(f"URL: {result.entry_id}")
    print("=" * 50)

C:\Users\prinnasit\AppData\Local\Temp\ipykernel_10248\2189810817.py:9: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


Title: Mathematical Logic in Computer Science
Authors: ['Assaf Kfoury']
Summary: The article retraces major events and milestones in the mutual influences
between mathematical logic and computer science since the 1950s.
Published: 2018-02-07 22:21:43+00:00
URL: http://arxiv.org/abs/1802.03292v1
Title: Ten Research Challenge Areas in Data Science
Authors: ['Jeannette M. Wing']
Summary: Although data science builds on knowledge from computer science, mathematics,
statistics, and other disciplines, data science is a unique field with many
mysteries to unlock: challenging scientific questions and pressing questions of
societal importance. This article starts with meta-questions about data science
as a discipline and then elaborates on ten ideas for the basis of a research
agenda for data science.
Published: 2020-01-27 21:39:57+00:00
URL: http://arxiv.org/abs/2002.05658v1
Title: Defining Data Science
Authors: ['Yangyong Zhu', 'Yun Xiong']
Summary: Data science is gaining more and more and w

In [ ]:
##api scopus

import requests
import csv

API_KEY = "c31125018f2d22db977ac3d0cb28dd37"
BASE_URL = "https://api.elsevier.com/content/search/scopus"
ABSTRACT_URL = "https://api.elsevier.com/content/abstract/scopus_id/"

# Function to fetch Scopus data based on a query
def fetch_scopus_data(query, count=25, max_results=100):
    headers = {
        "X-ELS-APIKey": API_KEY
    }

    params = {
        "query": query,
        "count": count,
        "start": 0,
        "view": "STANDARD",
        "sort": "citedby-count"
    }

    all_results = []
    fetched_results = 0

    while fetched_results < max_results:
        response = requests.get(BASE_URL, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            entries = data.get("search-results", {}).get("entry", [])
            for entry in entries:
                scopus_id = entry.get("dc:identifier", "").replace("SCOPUS_ID:", "")
                all_results.append({
                    "url": entry.get("prism:url"),
                    "identifier": scopus_id,
                    "eid": entry.get("eid"),
                    "title": entry.get("dc:title"),
                    "creator": parse_authors(entry.get("dc:creator")),
                    "publication_name": entry.get("prism:publicationName"),
                    "doi": entry.get("prism:doi"),
                    "cited_by_count": int(entry.get("citedby-count", 0)),
                    "affiliation": parse_affiliations(entry.get("affiliation", [])),
                    "cover_date": entry.get("prism:coverDate"),
                    "abstract": fetch_abstract(scopus_id),
                    "volume": entry.get("prism:volume"),
                    "open_access": entry.get("openaccess"),
                    "subtype": entry.get("subtypeDescription"),
                    "source_id": entry.get("source-id")
                })

            fetched_results += len(entries)
            if len(entries) < count:
                break
            params["start"] += count
        else:
            print(f"Failed to fetch data: {response.status_code} - {response.reason}")
            break

    return all_results

# Function to fetch abstract using Scopus ID
def fetch_abstract(scopus_id):
    if not scopus_id:
        return None

    url = f"{ABSTRACT_URL}{scopus_id}"
    headers = {
        "X-ELS-APIKey": API_KEY,
        "Accept": "application/json"
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return data.get("abstracts-retrieval-response", {}).get("coredata", {}).get("dc:description")
    else:
        print(f"Failed to fetch abstract for {scopus_id}: {response.status_code} - {response.reason}")
        return None

# Helper function to parse authors
def parse_authors(creator):
    if isinstance(creator, dict) and "author" in creator:
        authors = creator["author"]
        return "; ".join([f'{a["ce:indexed-name"]}' for a in authors])
    return creator

# Helper function to parse affiliations
def parse_affiliations(affiliations):
    return "; ".join(
        [f'{aff.get("affilname", "Unknown")}, {aff.get("affiliation-city", "Unknown")}, {aff.get("affiliation-country", "Unknown")}' for aff in affiliations]
    )

# Fetch data using a query
query = 'PUBYEAR > 2019 AND SUBJAREA(COMP) AND TITLE("computer")'
articles = fetch_scopus_data(query, count=10, max_results=100)

# Save results to CSV
output_filename = "scopus_data_with_abstracts2.csv"
with open(output_filename, "w", newline='', encoding="utf-8") as csv_file:
    fieldnames = [
        "url", "identifier", "eid", "title", "creator", "publication_name", "doi",
        "cited_by_count", "affiliation", "cover_date", "abstract", "volume", "open_access", 
        "subtype", "source_id"
    ]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    for article in articles:
        writer.writerow(article)

print(f"Fetched {len(articles)} articles with abstracts.")
print(f"Saved to {output_filename}.")


KeyboardInterrupt: 

In [ ]:

import os
import json
import pandas as pd

# กำหนดเส้นทางของโฟลเดอร์ที่เก็บไฟล์ JSON
folder_path = "Project/2020/"  # แก้ไขเป็นเส้นทางของโฟลเดอร์ที่คุณต้องการ

def process_file(file_path):
    try:
        # เปิดและโหลดไฟล์ JSON
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)

        # เข้าถึงข้อมูลหลัก
        abstracts_data = data.get("abstracts-retrieval-response", {})

        # ฟังก์ชันในการดึงข้อมูล
        def extract_data(abstracts_data):
            rows = []
            # # ตรวจสอบว่า data มีข้อมูลใน "affiliation" หรือไม่
            coredata = abstracts_data.get("coredata", {})
            subject_areas = abstracts_data.get("subject-areas", {}).get("subject-area", [])



            affiliations = abstracts_data.get("affiliation", [])
            # Get the first affiliation, defaulting to an empty dictionary if there are no affiliations
            first_affiliation = next(iter(affiliations), {})

            # Ensure first_affiliation is a dictionary before accessing .get() method
            first_affiliation_name = first_affiliation.get("affilname", "not have affiliation") if isinstance(first_affiliation, dict) else "not have affiliation"
            first_affiliation_city = first_affiliation.get("affiliation-city", "") if isinstance(first_affiliation, dict) else "not have affiliation"
            first_affiliation_country = first_affiliation.get("affiliation-country", "") if isinstance(first_affiliation, dict) else "not have affiliation"
            
            def check_first_affiliation_name(a):
                if a == "not have affiliation":
                    return affiliations.get("affilname", "")
                else:
                    return a
            def check_first_affiliation_city(a):
                if a == "not have affiliation":
                    return affiliations.get("affiliation-city", "")
                else:
                    return a
            def check_first_affiliation_country(a):
                if a == "not have affiliation":
                    return  affiliations.get("affiliation-country", "")
                else:
                    return a


            # ประมวลผลข้อมูลจาก subject-areas
            subjects_list = []
            for area in subject_areas:
                subject_details = {
                    "Name": area.get("$", ""),
                    "Abbreviation": area.get("@abbrev", ""),
                }
                subjects_list.append(subject_details)

            # สร้างข้อมูลแถว
            row = {
                "Title": coredata.get("dc:title", ""),
                "Abstract": coredata.get("dc:description", ""),
                "Publication Name": coredata.get("prism:publicationName", ""),
                "DOI": coredata.get("prism:doi", ""),
                "EID": coredata.get("eid", ""),
                "Cited By Count": coredata.get("citedby-count", ""),
                "Publication Date": coredata.get("prism:coverDate", ""),
                "Volume": coredata.get("prism:volume", ""),
                "ISSN": coredata.get("prism:issn", ""),
                "Article Number": coredata.get("article-number", ""),
                "Publisher": coredata.get("dc:publisher", ""),
                "Open Access": coredata.get("openaccess", ""),
                "Open Access Flag": coredata.get("openaccessFlag", ""),
                "Subtype Description": coredata.get("subtypeDescription", ""),
                "Source ID": coredata.get("source-id", ""),
                "First Affiliation Name": check_first_affiliation_name(first_affiliation_name),
                "First Affiliation City": check_first_affiliation_city(first_affiliation_city),
                "First Affiliation Country": check_first_affiliation_country(first_affiliation_country),
                "Subject Areas": subjects_list,
                "Scopus URL": next((link["@href"] for link in coredata.get("link", []) if link["@rel"] == "scopus"), "")
            }

            rows.append(row)
            return rows

        return extract_data(abstracts_data)

    except Exception as e:
        print(f"error in single file processing{file_path}: {e}")
        return []

# วนลูปเพื่อประมวลผลไฟล์ทั้งหมดในโฟลเดอร์
all_rows = []
if os.path.exists(folder_path) and os.path.isdir(folder_path):
    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Check if the path is a file (and not a directory)
        if os.path.isfile(file_path):
            try:
                # Process the file and extend the results to all_rows
                all_rows.extend(process_file(file_path))
            except Exception as e:
                print(f"Error processing file {filename}: {e}")
else:
    print(f"The folder path '{folder_path}' does not exist or is not a directory.")



# สร้าง DataFrame และบันทึกข้อมูลลงใน CSV เดียว
df = pd.DataFrame(all_rows)
output_csv = "2020_all_file.csv"  # ชื่อไฟล์ CSV ที่ต้องการ
df.to_csv(output_csv, index=False, encoding='utf-8')
print(f"CSV file saved as {output_csv}")


CSV file saved as 2020_all_file.csv


In [56]:
##this is for test folder 

import os
import json
import pandas as pd

# กำหนดเส้นทางของโฟลเดอร์ที่เก็บไฟล์ JSON
folder_path = "Project/2023/"  # แก้ไขเป็นเส้นทางของโฟลเดอร์ที่คุณต้องการ

def process_file(file_path):
    try:
        # เปิดและโหลดไฟล์ JSON
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)

        # เข้าถึงข้อมูลหลัก
        abstracts_data = data.get("abstracts-retrieval-response", {})

        # ฟังก์ชันในการดึงข้อมูล
        def extract_data(abstracts_data):
            rows = []
            # # ตรวจสอบว่า data มีข้อมูลใน "affiliation" หรือไม่
            coredata = abstracts_data.get("coredata", {})
            subject_areas = abstracts_data.get("subject-areas", {}).get("subject-area", [])
            authors_data = abstracts_data.get("authors", {}).get("author", [])


            affiliations = abstracts_data.get("affiliation", [])
            # Get the first affiliation, defaulting to an empty dictionary if there are no affiliations
            first_affiliation = next(iter(affiliations), {})

            authors = [
            f"{author.get('ce:given-name', '')} {author.get('ce:surname', '')} ({author.get('ce:indexed-name', '')})"
            for author in authors_data
            ]

            # Ensure first_affiliation is a dictionary before accessing .get() method
            first_affiliation_name = first_affiliation.get("affilname", "not have affiliation") if isinstance(first_affiliation, dict) else "not have affiliation"
            first_affiliation_city = first_affiliation.get("affiliation-city", "") if isinstance(first_affiliation, dict) else "not have affiliation"
            first_affiliation_country = first_affiliation.get("affiliation-country", "") if isinstance(first_affiliation, dict) else "not have affiliation"
            
            def check_first_affiliation_name(a):
                if a == "not have affiliation":
                    return affiliations.get("affilname", "")
                else:
                    return a
            def check_first_affiliation_city(a):
                if a == "not have affiliation":
                    return affiliations.get("affiliation-city", "")
                else:
                    return a
            def check_first_affiliation_country(a):
                if a == "not have affiliation":
                    return  affiliations.get("affiliation-country", "")
                else:
                    return a


            # ประมวลผลข้อมูลจาก subject-areas
            subjects_list = []
            for area in subject_areas:
                subject_details = {
                    "Name": area.get("$", ""),
                    "Abbreviation": area.get("@abbrev", ""),
                }
                subjects_list.append(subject_details)

            # สร้างข้อมูลแถว
            row = {
                "Title": coredata.get("dc:title", ""),
                "Abstract": coredata.get("dc:description", ""),
                "Publication Name": coredata.get("prism:publicationName", ""),
                "DOI": coredata.get("prism:doi", ""),
                "EID": coredata.get("eid", ""),
                "Author": "; ".join(authors),
                "Cited By Count": coredata.get("citedby-count", ""),
                "Publication Date": coredata.get("prism:coverDate", ""),
                "Volume": coredata.get("prism:volume", ""),
                "ISSN": coredata.get("prism:issn", ""),
                "Article Number": coredata.get("article-number", ""),
                "Publisher": coredata.get("dc:publisher", ""),
                "Open Access": coredata.get("openaccess", ""),
                "Open Access Flag": coredata.get("openaccessFlag", ""),
                "Subtype Description": coredata.get("subtypeDescription", ""),
                "Source ID": coredata.get("source-id", ""),
                "First Affiliation Name": check_first_affiliation_name(first_affiliation_name),
                "First Affiliation City": check_first_affiliation_city(first_affiliation_city),
                "First Affiliation Country": check_first_affiliation_country(first_affiliation_country),
                "Subject Areas": subjects_list,
                "Scopus URL": next((link["@href"] for link in coredata.get("link", []) if link["@rel"] == "scopus"), "")
            }

            rows.append(row)
            return rows

        return extract_data(abstracts_data)

    except Exception as e:
        print(f"error in single file processing{file_path}: {e}")
        return []

# วนลูปเพื่อประมวลผลไฟล์ทั้งหมดในโฟลเดอร์
all_rows = []
if os.path.exists(folder_path) and os.path.isdir(folder_path):
    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Check if the path is a file (and not a directory)
        if os.path.isfile(file_path):
            try:
                # Process the file and extend the results to all_rows
                all_rows.extend(process_file(file_path))
            except Exception as e:
                print(f"Error processing file {filename}: {e}")
else:
    print(f"The folder path '{folder_path}' does not exist or is not a directory.")



# สร้าง DataFrame และบันทึกข้อมูลลงใน CSV เดียว
df = pd.DataFrame(all_rows)
output_csv = "2023_all_file.csv"  # ชื่อไฟล์ CSV ที่ต้องการ
df.to_csv(output_csv, index=False, encoding='utf-8')
print(f"CSV file saved as {output_csv}")


CSV file saved as 2023_all_file.csv
